## Лабораторная работа 4

1. Выделить таргетивную метку - популярности поста (по пороговому значению лайков\просмотров и репостов)
2. Разделить данные на обучающую, валидационную и тестовую выборку.
3. Исходя из вашей бд подумать какие знаки пунктуации и стоп-слова необходимо удалить, помимо базовых. Удалять ли смайлики, эмотиконы, ссылки и пр.?
4. Составить функцию предобработки текстов на основе обучающего набора. Применить эту функцию к обучающему, валидациоонному и тестовому набору.
5. Добавить колонку в DataFrame с получившимся предобработанным текстом
6. Сохранить DataFrame в отдельном файле csv (Важно не пересохранить в предыдущий файл)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/My Drive/data_base.csv')
df

,id_post,date,text,views,reactions,comments
0,24383,2025-04-19 06:00:24+00:00,Universal Pictures анонсировала спортивный хор...,34219,247,25
1,24382,2025-04-18 20:24:57+00:00,"Народ, видос долго заливался и обрабатывался! ...",82358,749,39
2,24381,2025-04-18 17:59:01+00:00,**Tesla обвиняется в чрезмерной накрутке пробе...,88704,916,33
3,24380,2025-04-18 17:01:37+00:00,**Google добавит красок в чистый Android 16**\...,82788,477,78
4,24378,2025-04-18 16:01:12+00:00,**Новая игра от авторов FTL и Into the Breach ...,78271,417,11
...,...,...,...,...,...,...
15508,9,2017-06-01 07:11:25+00:00,Камера у iPhone 7 Plus - огонь. Но нет предела...,10717,103,0
15509,7,2017-05-31 18:32:23+00:00,И сюда же линк на SoundCloud https://soundclou...,13767,117,0
15510,6,2017-05-31 18:31:45+00:00,А пока сводка коротеньких новостей: завтра зап...,19377,130,0
15511,5,2017-05-31 18:29:09+00:00,Добрейшего вечерочка! Тут будет вся подноготна...,19844,351,0


Выделяем таргетивную метку

In [ ]:
# проведем оценку вовлеченности
df['engagement'] = (df['reactions'] + df['comments']) / df['views']
# если оценка вовлеченности больше 2%, то пост считается популярным
df['target'] = (df['engagement']>0.02).astype(bool)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15513 entries, 0 to 15512
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id_post     15513 non-null  int64  
 1   date        15513 non-null  object 
 2   text        15513 non-null  object 
 3   views       15513 non-null  int64  
 4   reactions   15513 non-null  int64  
 5   comments    15513 non-null  int64  
 6   engagement  15513 non-null  float64
 7   target      15513 non-null  bool   
dtypes: bool(1), float64(1), int64(4), object(2)
memory usage: 863.6+ KB


Разделяем данные на обучающую, валидационную и тестовую выборку.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# выделяем из всех данных обучающую и тестовую выборку
df_train, df_test = train_test_split(df, test_size = 0.15, random_state = 42, stratify = df['target'])

In [ ]:
# выделяем из обучающих данных обучающую и валидационную выборку
df_train, df_val = train_test_split(df_train, test_size = 0.15, random_state = 42, stratify = df_train['target'])

Обработка

In [ ]:
df['text']

,text
0,Universal Pictures анонсировала спортивный хор...
1,"Народ, видос долго заливался и обрабатывался! ..."
2,**Tesla обвиняется в чрезмерной накрутке пробе...
3,**Google добавит красок в чистый Android 16**\...
4,**Новая игра от авторов FTL и Into the Breach ...
...,...
15508,Камера у iPhone 7 Plus - огонь. Но нет предела...
15509,И сюда же линк на SoundCloud https://soundclou...
15510,А пока сводка коротеньких новостей: завтра зап...
15511,Добрейшего вечерочка! Тут будет вся подноготна...


In [ ]:
df['text'][0]

'Universal Pictures анонсировала спортивный хоррор **HIM** от Джордана Пила!\n\n**«Его»** срежессирует Джастин Типпинг, премьера 19 сентября 2025 года.'

In [ ]:
df['text'][15508]

'Камера у iPhone 7 Plus - огонь. Но нет предела совершенству. На Kickstarter собрали средства на проект 6-линзового чехла для телефона под лознугом «нужно больше зума!» обещают, что он появится в продаже осенью за $99.\nhttps://wylsa.com/iphone-7-plus-case-with-6-lenses/'

In [ ]:
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('russian')

In [ ]:
stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [ ]:
# удаляем слова из стоп-слов
custom_stopwords = set(stopwords)
stopwords = custom_stopwords.difference({'хорошо', 'лучше', 'нельзя'})
# добавим в стоп-слова
stopwords.update({'это', 'лол', 'ага'})
stopwords

{'а',
 'ага',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лол',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так',
 '

In [ ]:
# функция для удаления слов с повторяющимися символами (3 и больше)
def duplicate_characters(word):
  return re.search(r'(.)\1{2,}', word) is not None

In [ ]:
!pip install pymorphy3 > None

In [ ]:
from pymorphy3 import MorphAnalyzer

In [ ]:
morph = MorphAnalyzer()

In [ ]:
# функция предобработки
def preprocess(text):
  if pd.isna(text):
        return ""

  # понижаем регистр
  text = text.lower()
  # удаляем подписи перед ссылками
  text = re.sub(r'^\s*[«"]?(youtube|yt|vk(?:\s+видео)?)["»]?:\s*https?:\/\/\S+', '', text)
  # удаляем markdown-ссылки
  text = re.sub(r'\[.*?\]\(https?:\/\/[^\s)]+\)', '', text)
  # удаляем остальные ссылки и упоминания пользователей
  text = re.sub(r'http\S+|www\S+|https\S+', '', text)
  text = re.sub(r'tg://user\?id=\d+', '', text)
  # удаляем почты
  text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '', text)
  # удаляем все символы, кроме английских и русских букв
  text = re.sub(r'[^a-zа-яё]', " ", text)
  tokens = word_tokenize(text, language='russian')
  # удаляем стоп-слова
  tokens = [i for i in tokens if i not in stopwords and not duplicate_characters(i)]
  # леммантизируем текст
  lemms = [morph.parse(i)[0].normal_form for i in tokens]

  return ' '.join(lemms)

In [ ]:
df_train['processed_text'] = df_train['text'].apply(preprocess)
df_val['processed_text'] = df_val['text'].apply(preprocess)
df_test['processed_text'] = df_test['text'].apply(preprocess)


In [ ]:
df_train

,id_post,date,text,views,reactions,comments,engagement,target,processed_text
4754,15223,2024-03-14 11:59:58+00:00,Продолжаем наш совместный с @MosGuru розыгрыш!...,226362,1372,85,0.006437,False,продолжать наш совместный mosguru розыгрыш он ...
3587,17189,2024-06-11 09:50:39+00:00,"Клавиатуры добавили только тех стран, в которы...",167889,613,57,0.003991,False,клавиатура добавить тот страна который продава...
8867,7599,2022-12-30 12:15:33+00:00,**В чипе A17 для iPhone 15 Apple сфокусируется...,182941,2289,0,0.012512,False,чип a iphone apple сфокусироваться энергоэффек...
11790,4092,2021-07-01 08:52:18+00:00,Вышла публичная бета iOS 15. Ставить на основн...,20043,9,0,0.000449,False,выйти публичный бета ios ставить основный устр...
9499,6762,2022-09-17 16:51:50+00:00,Вот так делаешь. И eSIM улетела на новый iPhone.,401874,5783,0,0.014390,False,делать esim улететь новый iphone
...,...,...,...,...,...,...,...,...,...
3371,17583,2024-07-02 16:20:42+00:00,**В Техасе есть точная копия стамбульской трас...,191088,1254,79,0.006976,False,техас точный копия стамбульский трасса формула...
3,24380,2025-04-18 17:01:37+00:00,**Google добавит красок в чистый Android 16**\...,82788,477,78,0.006704,False,google добавить краска чистый android новый бе...
8411,8339,2023-02-28 11:03:01+00:00,**Redmi **[**показал**](https://weibo.com/3021...,165714,1507,0,0.009094,False,redmi зарядка вт нуль пять минута xiaomi разра...
10939,5016,2021-12-10 09:03:46+00:00,Мода на кроссовки в стиле тапочек набирает обо...,16392,21,0,0.001281,False,мода кроссовок стиль тапочки набирать оборот n...


In [ ]:
df_train['processed_text'] = df_train['processed_text'].fillna('')
df_val['processed_text'] = df_val['processed_text'].fillna('')
df_test['processed_text'] = df_test['processed_text'].fillna('')

In [ ]:
df_train['processed_text'].isna().sum()

np.int64(0)

In [ ]:
# сохраняем выборки
df_train.to_csv('/content/drive/My Drive/data_train.csv', index=False)
df_val.to_csv('/content/drive/My Drive/data_val.csv', index=False)
df_test.to_csv('/content/drive/My Drive/data_test.csv', index=False)

In [ ]:
df_final = pd.concat([df_train, df_val, df_test])

In [ ]:
df_final.to_csv('/content/drive/My Drive/data_processed.csv', index=False)